# Import

# Pre- req install surprise:
# conda install -c  conda-forge scikit-surprise


# USER - USER Collaborative Filtering

## Libraries:

In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, KFold
from surprise import accuracy


# Read Data

In [25]:
def readData():
    mov = pd.read_csv('C://Users//manis//Downloads//UML_Project//ml-latest-small//movies.csv',encoding="Latin1")
    rat = pd.read_csv('C://Users//manis//Downloads//UML_Project//ml-latest-small//ratings.csv')
    mean = rat.groupby(by="userId",as_index=False)['rating'].mean()
    avg = pd.merge(rat,mean,on='userId')
    avg['avg_rating']=avg['rating_x']-avg['rating_y']
    
    
    return avg, mov, mean, rat


# Pre-Process Data

In [26]:
def preprocessData(avg, mov):
    check = pd.pivot_table(avg,values='rating_x',index='userId',columns='movieId')
    final=pd.pivot_table(avg,values='avg_rating',index='userId',columns='movieId')
    movie = final.fillna(final.mean(axis=0))
    user = final.apply(lambda row: row.fillna(row.mean()), axis=1)
    cos = cosine_similarity(user)
    np.fill_diagonal(cos, 0 )
    usersim = pd.DataFrame(cos,index=user.index)
    usersim.columns=user.index
    cos = cosine_similarity(movie)
    np.fill_diagonal(cos, 0 )
    moviesim = pd.DataFrame(cos,index=movie.index)
    moviesim.columns = user.index
    
    return usersim, moviesim, movie, user, check

# Find Nearest Neighbour

In [27]:
def findneighbour(data,n):
    data = data.apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:n].index, index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return data

# Find User User common movies

In [28]:
def user_user( user1, user2,avg,mov ):
    common_movies = avg[avg.userId == user1].merge(
    avg[avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( mov, on = 'movieId' )

In [29]:
def testUserUser(user1, user2):
    avg,  mov, mean,rat  = readData()
    usersim, moviesim, movie, user, check = preprocessData(avg, mov)
    m = findneighbour(moviesim,50)
    u = findneighbour(usersim,50)
    data = []
    a = user_user(user1,user2,avg,mov)
    a = a.loc[ : , ['rating_x_x','rating_x_y','title']]
    data.append(a['title'])
    return data

In [7]:
user1 = int(input("Enter the user-id 1 to whom you want to recommend : "))
user2 = int(input("Enter the user-id 2 to whom you want to recommend :"))
testUserUser(user1, user2)

Enter the user-id 1 to whom you want to recommend : 50
Enter the user-id 2 to whom you want to recommend :85


[0    Mars Attacks! (1996)
 Name: title, dtype: object]

In [30]:
def user_item_item(user,check,m,movie, mean,moviesim,avg,mov, item):
    avg = avg.astype({"movieId": str})
    mov_user = avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))
    user_movies = [check.columns[check[check.index==user].notna().any()]]
    
    user_movies_20 = [user_movies[0][0:int(len(user_movies[0])*0.7)]]
    mindex = (m[m.index==user].values).squeeze().tolist()
    imovies = mov_user[mov_user.index.isin(mindex)]
    stri = ','.join(imovies.values)
    simusermovies = stri.split(',')
    movieRecommend = list(set(simusermovies)-set(list(map(str, user_movies_20))))
    movieRecommend = list(map(int, movieRecommend))
    scores = []
    noOfRecommendation = len(user_movies[0])
    item = user_movies[0][0]
    for item in movieRecommend:
        moviess = ( movie.loc[:,item])[( movie.loc[:,item]).index.isin(mindex)]
        notNullMov = moviess[moviess.notnull()]
        index = notNullMov.index.values.squeeze().tolist()
        corr = moviesim.loc[user,index]
        co = pd.concat([notNullMov, corr], axis=1)
        co.columns = ['avg_score','correlation']
        co['score']=co.apply(lambda x:x['avg_score'] * x['correlation'],axis=1)
        score = (mean.loc[mean['userId'] == user,'rating'].values[0]) + ((co['score'].sum())/co['correlation'].sum())
        scores.append(score)
    data = pd.DataFrame({'movieId':movieRecommend,'score':scores})
    
    userData = pd.DataFrame({'movieId':user_movies[0]})
    tile =  userData.merge(mov, how='inner', on='movieId')
    tiles = tile.title.tolist()

    
    
    recommend = data.sort_values(by='score',ascending=False).head(noOfRecommendation)
    name = recommend.merge(mov, how='inner', on='movieId')
    names = name.title.tolist()
    genre = name.genres.tolist()
    return names, genre,user_movies,tiles

In [11]:
def user_item_movieRecommend():
    avg,  mov, mean,rat  = readData()
    usersim, moviesim, movie, user, check = preprocessData(avg, mov)
    m = findneighbour(moviesim,50)
    u = findneighbour(usersim,50)
    
    user = int(input("Enter the user-id 1 to whom you want to recommend : "))

    predicted_movies, genre,user_movies,tiles = user_item_item(user,check,m,movie, mean,moviesim,avg,mov, 100)    
    print(" ")
    print("Recommendation for User 85 using User-User Collaborative")
    print(" ")
    pred = [dict()]
    for i in range(len(predicted_movies)):
        print(predicted_movies[i])
    
#     print(user_movies)    
    a = 0
    for i in predicted_movies:
        if i in tiles:
            a += 1
    print("Precision at 100: ",  a*100/len(tiles))

In [12]:
user_item_movieRecommend()

Enter the user-id 1 to whom you want to recommend : 85
 
Recommendation for User 85 using User-User Collaborative
 
Three Billboards Outside Ebbing, Missouri (2017)
Return of Martin Guerre, The (Retour de Martin Guerre, Le) (1982)
Streetcar Named Desire, A (1951)
Wristcutters: A Love Story (2006)
Tekkonkinkreet (Tekkon kinkurÃ®to) (2006)
Ghost in the Shell (KÃ´kaku kidÃ´tai) (1995)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
Hoop Dreams (1994)
Perfect Blue (1997)
Shawshank Redemption, The (1994)
Crumb (1994)
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
Harold and Maude (1971)
Planet Earth (2006)
Evil Dead II (Dead by Dawn) (1987)
Apocalypse Now (1979)
Boogie Nights (1997)
Usual Suspects, The (1995)
Princess Bride, The (1987)
Monty Python and the Holy Grail (1975)
Goodfellas (1990)
Casablanca (1942)
Godfather, The (1972)
Mean Streets (1973)
Brazil (1985)
Seven Samurai (Shichinin no samurai) (1954)
Star Wars: Episode IV - A New Hope (1977)
Godfather: Part

# Item - Item

In [31]:
def item_item(user1):
    avg,  mov, mean,rat  = readData()
    ratings = pd.merge(mov, rat)
    user = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
    corr = user.corr()
    RatingsUser = user.loc[user1].dropna()
    print(RatingsUser)
    simUser = pd.Series()
    for i in range(0, len(RatingsUser.index)):
        simMovies = (corr[RatingsUser.index[i]].dropna()).map(lambda x: x * RatingsUser[i])
        simUser = simUser.append(simMovies)
    simUser.sort_values(inplace = True, ascending = False)
    data = simUser.head(len(RatingsUser))
    
    a = 0
    for i in RatingsUser:
        if i in data:
            a += 1
        print("Precision at 100: ",  a*100/len(data))

    return data

In [32]:
user1 = int(input("Enter the user id to whom you want to recommend : "))
data = item_item(user1)


Enter the user id to whom you want to recommend : 85
title
Amistad (1997)                                        4.0
Apostle, The (1997)                                   4.0
Bent (1997)                                           2.0
Broken English (1996)                                 1.0
Career Girls (1997)                                   4.0
Contact (1997)                                        3.0
English Patient, The (1996)                           4.0
Entertaining Angels: The Dorothy Day Story (1996)     5.0
Full Monty, The (1997)                                5.0
Good Will Hunting (1997)                              3.0
Ice Storm, The (1997)                                 4.0
Kolya (Kolja) (1996)                                  4.0
Kundun (1997)                                         4.0
Lamerica (1994)                                       5.0
Mars Attacks! (1996)                                  1.0
Midnight in the Garden of Good and Evil (1997)        2.0
Mother (1996)

In [ ]:
user_user